In [ ]:
import pandas as pd
from tqdm import tqdm

In [ ]:
!pip install huggingface_hub[hf_transfer]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 31.1 MB/s eta 0:00:00


In [ ]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00


In [ ]:
!export HF_HUB_ENABLE_HF_TRANSFER=1

In [ ]:
from datasets import load_dataset

In [ ]:
from huggingface_hub import InferenceClient

In [ ]:
client = InferenceClient(api_key="")

In [ ]:
#Carga del dataset 100k
ds = load_dataset("argilla/FinePersonas-v0.1-clustering-100k", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/37.1k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/350M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/350M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
df = pd.DataFrame({
    'id': ds['id'],
    'persona': ds['persona'],
    'cluster_label': ds['cluster_label'],
    'summary_label': ds['summary_label']
})

In [ ]:
df.head()

,id,persona,cluster_label,summary_label
0,<urn:uuid:4cda0919-3bce-43ce-aa46-6c205b242490>,A geography teacher or high school educator fo...,-1,"""None"""
1,<urn:uuid:2bdad496-891e-44f0-8041-49f9a381a88a>,A meteorologist or a radar technician dealing ...,0,"[""Environmental"", ""Scientific"", ""Professional""]"
2,<urn:uuid:9e0159f8-2478-4fc2-818a-e325f1b1e9ac>,A public health specialist focused on childhoo...,0,"[""Environmental"", ""Scientific"", ""Professional""]"
3,<urn:uuid:b30c1d38-869e-495d-a574-8b702f29522c>,An English as a second language (ESL) teacher ...,1,"[""Education"", ""Academia"", ""Specialized Experti..."
4,<urn:uuid:fdddfb80-360e-4cd9-8cbd-583976fe9520>,An academic or educator in the field of econom...,2,"[""Academic"", ""Economics"", ""Finance""]"


In [ ]:
sample = df['persona'].sample(15)

In [ ]:
sample

,persona
83580,A middle school or high school social studies ...
43323,An elementary school teacher focused on hands-...
25122,A physics or engineering student/researcher en...
90812,A paleographer or biblical manuscript scholar ...
56929,A high school history student researching or w...
65692,A geneticist or molecular biologist specializi...
26758,A user experience (UX) designer or researcher ...
40495,A paleontologist specializing in Cambrian-era ...
36664,A biological anthropologist specializing in pr...
36875,A historian with an interest in penology or a ...


# Parametrización

Reacción, Interés, preocupación, nivel de aceptación, jiribilla (tomar ventaja en cuestiones ilegales, inmorales,de legalidad dudosa o que responde a intereses individuales)

In [ ]:
situacion_a_observar = "Reaction"

In [ ]:
project = "Construction of a new airport with a massive budget"

In [ ]:
PROMPT_TEMPLATE_1 = f"""
Based on the following persona, describe the {situacion_a_observar}
if we implement the following project that is a political implementation: {project}, and he belongs to a minority population
Example: 'Good reaction, and my observations are that they need to focus on...'

"""

In [ ]:
sample = pd.DataFrame(sample, columns=['persona'])

In [ ]:
print(PROMPT_TEMPLATE_1)

 
Based on the following persona, describe the Reaction 
if we implement the following project that is a political implementation: Construction of a new airport with a massive budget, and he belongs to a minority population
Example: 'Good reaction, and my observations are that they need to focus on...'




In [ ]:
def obtener_respuesta(prompt):
    output = ""
    for message in client.chat_completion(
        model="meta-llama/Llama-3.1-8B-Instruct",
        messages=[
            {"role": "system", "content": "You're a simulated person that responds in Spanish in 100 words, straight to the point"},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,
        stream=True
    ):
        if message.choices[0].delta.content is not None:
            output += message.choices[0].delta.content
    return output

In [156]:
def procesar_descripciones(df, prompt_template):
    resultados = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Procesando personas"):
        persona = row['persona']
        prompt = prompt_template.format(persona=persona)

        try:
            output = obtener_respuesta(prompt)
            resultados.append({'persona': persona, 'output': output})
        except Exception as e:
            print(f"Error procesando persona: {persona}. Error: {str(e)}")
            resultados.append({'persona': persona, 'output': 'Error en el procesamiento'})

    return pd.DataFrame(resultados)

In [159]:
# Uso de la función
prompt_template = """
Simulate how the following consumer segment would react to the launch of a new wearable technology product
(e.g., a smartwatch) that monitors health, focusing on disease prevention and wellness:
{persona}
"""

# Asumiendo que ya tienes tu DataFrame 'df' con la columna 'persona'
df_resultados = procesar_descripciones(sample, prompt_template)

Procesando personas: 100%|██████████| 15/15 [00:00<00:00, 21.74it/s]


In [160]:
df_resultados.head()

,persona,output
0,A science teacher or educator who guides stude...,El profesor de ciencias experimentaría con la ...
1,A grade school educator or homeschooling paren...,"""La verdad es que no creo que nuestras necesid..."
2,A wildlife biologist or naturalist specializin...,Como biólogo de vida silvestre o naturalista q...
3,A physics educator specializing in explaining ...,"""¡Ya verás lo que esto va a cambiar la conducc..."
4,A TEFL teacher and educator focused on the nee...,El lanzamiento de este nuevo reloj inteligente...
